In [1]:
import pandas as pd
import os
import glob

# CSVファイルが格納されているディレクトリのパス
csv_dir = "csv_exports"  # 必要に応じて変更
csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))

# 各テーブルを格納する辞書
tables = {}

# 各CSVファイルをデータフレームとして読み込む
for file in csv_files:
    table_name = os.path.basename(file).replace(".csv", "")  # ファイル名をテーブル名として使用
    df = pd.read_csv(file, encoding="utf-8")
    tables[table_name] = df
    print(f"テーブル '{table_name}' をインポートしました。")

# 必要なテーブルの取得
required_tables = ["customer", "rental", "inventory", "film_actor", "actor"]
missing_tables = [t for t in required_tables if t not in tables]

# 欠落しているテーブルを確認
if missing_tables:
    print(f"以下のテーブルが見つかりませんでした: {', '.join(missing_tables)}")
else:
    print("すべてのテーブルが正しくロードされました。")

    # 各テーブルのカラム情報を表示
    for table_name in required_tables:
        print(f"\n=== {table_name} のカラム ===")
        print(tables[table_name].columns.tolist())

    # 必要なテーブルを取得
    customer_df = tables["customer"]
    rental_df = tables["rental"]
    inventory_df = tables["inventory"]
    film_actor_df = tables["film_actor"]
    actor_df = tables["actor"]

    # データ型を適切に変換（数値カラムを数値型に変換）
    for df in [customer_df, rental_df, inventory_df, film_actor_df, actor_df]:
        for col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col])
            except:
                pass

    # `last_update` の重複を避けるために削除
    drop_columns = ["last_update"]
    for df in [customer_df, rental_df, inventory_df, film_actor_df, actor_df]:
        df.drop(columns=[col for col in drop_columns if col in df.columns], inplace=True, errors='ignore')

    # テーブルの結合（suffixes で `_y` を付けることで `last_update` の重複を回避）
    merged_df = (
        rental_df
        .merge(customer_df, on="customer_id", how="inner")  # 顧客情報
        .merge(inventory_df, on="inventory_id", how="inner")  # 在庫情報
        .merge(film_actor_df, on="film_id", how="inner")  # 映画と俳優の関係
        .merge(actor_df, on="actor_id", how="inner", suffixes=("_actor", "_customer"))  # 俳優情報
    )

    # `last_update_y` の削除
    if "last_update_y" in merged_df.columns:
        merged_df.drop(columns=["last_update_y"], inplace=True)

    # 欠損データを削除（NaNを含む行を除外）
    merged_df.dropna(inplace=True)

    # 顧客ごとに好む俳優を集計
    result_df = (
        merged_df.groupby(["customer_id", "first_name_customer", "last_name_customer", "actor_id", "first_name_actor", "last_name_actor"])
        .size()
        .reset_index(name="rental_count")
        .sort_values(["customer_id", "rental_count"], ascending=[True, False])
    )

    # 結果の表示
    print("\n=== 顧客ごとの好む俳優ランキング（TOP10） ===")
    print(result_df.head(10))  # 上位10行を表示


以下のテーブルが見つかりませんでした: customer, rental, inventory, film_actor, actor
